<a href="https://colab.research.google.com/github/killerkiemklan/Statistics4DS_Financial_analysis/blob/main/Stats4DS_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Research Question:** How do technical metrics (e.g., moving averages, RSI) and risk metrics (e.g., Beta, Sharpe ratio, Volatility) influence key financial performance indicators (Price, Debt-to-Equity Ratio, Profit Margin, and Return on Equity) for major technology companies?

**This study investigates the influence of technical metrics (e.g., MA10, MA20, RSI) and risk metrics (e.g., Beta, Sharpe ratio, Volatility) on key financial performance indicators (Price, Debt-to-Equity Ratio, Profit Margin, and ROE) for major technology companies. Using panel data from 2010 to 2019, we employ random effects models to analyze the relationships. The results show that technical and risk metrics are highly effective in explaining Price, but have limited explanatory power for Debt-to-Equity, Profit Margin, and ROE. The findings suggest that risk metrics, such as Beta and Volatility, are key drivers of Price, while other factors may be more important for explaining Debt-to-Equity, Profit Margin, and ROE.**

**Hypotheses:**

Technical metrics (e.g., MA10, MA20, RSI) significantly influence Price, Debt-to-Equity, Profit Margin, and ROE.

Risk metrics (e.g., Beta, Sharpe ratio, Volatility) significantly influence Price, Debt-to-Equity, Profit Margin, and ROE.

The dataset includes financial and market data for major technology companies (e.g., Apple, Google, Microsoft, Amazon, Meta, Tesla) and are grabbed from Alphavantage API

In [1]:
# PHASE 0: CRITICAL LIBRARY IMPORTATION WITH QUANTUM-LEVEL PRECISION
if (!require(pacman)) install.packages("pacman")
pacman::p_load(
  tidyverse,  # For data manipulation - CRITICAL
  plm,        # For panel analysis - ABSOLUTELY ESSENTIAL
  lmtest,     # For statistical validation - MATHEMATICALLY CRUCIAL
  car,        # For diagnostics - RELATIVISTICALLY IMPORTANT
  tseries,    # For time series components - QUANTUM NECESSARY
  gridExtra,  # For visualization arrangement - CRITICALLY NEEDED
  corrplot,   # For correlation analysis - ASTRONOMICALLY VITAL
  grid,       # For advanced plotting - FUNDAMENTALLY REQUIRED
  git2r,
  scales,
  reshape2,
  stargazer,
  viridis,
  sandwich,   # Robust standard errors
  broom,
  kableExtra

)

# Set warnings as errors for stricter debugging
options(warn = 2)

suppressPackageStartupMessages({
  library(plm)        # Panel data econometrics
  library(lmtest)     # Diagnostic testing
  library(ggplot2)    # Data visualization
  library(dplyr)      # Data manipulation
  library(tidyr)      # Data transformation
  library(gridExtra)  # Plot arrangement
  library(corrplot)   # Correlation visualization
  library(sandwich)   # Robust standard errors
  library(broom)      # For augment function
})

Loading required package: pacman

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘pacman’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘miscTools’, ‘rbibutils’, ‘bdsmatrix’, ‘collapse’, ‘zoo’, ‘sandwich’, ‘lmtest’, ‘maxLik’, ‘Rdpack’, ‘Formula’



plm installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘cowplot’, ‘Deriv’, ‘microbenchmark’, ‘numDeriv’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘lme4’



car installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘xts’, ‘TTR’, ‘quadprog’, ‘quantmod’



tseries installed

Installing package in

In [2]:

# Clone the repository using HTTPS
git2r::clone("https://github.com/killerkiemklan/Statistics4DS_Financial_analysis.git", "Statistics4DS_Financial_analysis2")

cloning into 'Statistics4DS_Financial_analysis2'...
Receiving objects:   3% (1/26),    9 kb
Receiving objects:  11% (3/26),    9 kb
Receiving objects:  23% (6/26),    9 kb
Receiving objects:  34% (9/26),    9 kb
Receiving objects:  42% (11/26),   26 kb
Receiving objects:  53% (14/26),  242 kb
Receiving objects:  61% (16/26),  242 kb
Receiving objects:  73% (19/26),  242 kb
Receiving objects:  84% (22/26),  416 kb
Receiving objects:  92% (24/26),  416 kb
Receiving objects: 100% (26/26),  416 kb, done.


Local:    main /content/Statistics4DS_Financial_analysis2
Remote:   main @ origin (https://github.com/killerkiemklan/Statistics4DS_Financial_analysis.git)
Head:     [fa77c97] 2025-01-09: Add files via upload

In [3]:
# --------------------------
# Phase 1: Data Acquisition
# --------------------------
# Load financial data for Amazon, Google, Microsoft, Apple, Facebook, and Tesla
tryCatch({
  amzn <- read.csv("/content/Statistics4DS_Financial_analysis2/AMZN_combined_metrics_quarterly.csv")
  goog <- read.csv("/content/Statistics4DS_Financial_analysis2/GOOG_combined_metrics_quarterly.csv")
  msft <- read.csv("/content/Statistics4DS_Financial_analysis2/MSFT_combined_metrics_quarterly.csv")
  aapl <- read.csv("/content/Statistics4DS_Financial_analysis2/AAPL_combined_metrics_quarterly.csv")  # Apple
  meta <- read.csv("/content/Statistics4DS_Financial_analysis2/META_combined_metrics_quarterly.csv")      # Facebook (Meta)
  tsla <- read.csv("/content/Statistics4DS_Financial_analysis2/TSLA_combined_metrics_quarterly.csv")  # Tesla
}, error = function(e) {
  stop("Data loading failed: ", e$message)
})

In [ ]:
# --------------------------
# Phase 2: Data Transformation
# --------------------------
# Combine data from all companies into a single dataset
panel_data <- rbind(
  transform(amzn, company = "AMZN"),
  transform(goog, company = "GOOG"),
  transform(msft, company = "MSFT"),
  transform(aapl, company = "AAPL"),  # Add Apple
  transform(meta, company = "META"),  # Add Facebook (Meta)
  transform(tsla, company = "TSLA")   # Add Tesla
) %>%
  mutate(
    Date = as.Date(Date),  # Convert Date column to Date type
    # Log-transform financial metrics for normalization
    log_Price = log(pmax(Price, 0.01)),
    log_DebtToEquity = log(abs(DebtToEquity) + 1),
    log_ProfitMargin = log(abs(ProfitMargin) + 1),
    log_ROE = log(abs(ROE) + 1),
    # Engineer additional features
    MA_Ratio = MA10 / MA20,
    RSI_Normalized = (RSI - min(RSI)) / (max(RSI) - min(RSI)),
    Sharpe_Normalized = (Sharpe - min(Sharpe)) / (max(Sharpe) - min(Sharpe)),
    Volatility_Normalized = (Volatility - min(Volatility)) / (max(Volatility) - min(Volatility))
  ) %>%
  na.omit() %>%  # Remove rows with missing values
  arrange(company, Date)  # Sort by company and date

# Convert to panel data structure
pdata <- pdata.frame(panel_data, index = c("company", "Quarter"))


In [ ]:

# --------------------------
# Phase 4: Econometric Modeling
# --------------------------
# Step 1: Hausman Test to Compare Fixed and Random Effects Models
# Fit fixed effects models
fixed_price <- plm(log_Price ~ MA10 + MA20 + RSI_Normalized + Beta + Sharpe_Normalized + Volatility_Normalized,
                   data = pdata, model = "within")
fixed_debt <- plm(log_DebtToEquity ~ MA10 + MA20 + RSI_Normalized + Beta + Sharpe_Normalized + Volatility_Normalized,
                  data = pdata, model = "within")
fixed_profit <- plm(log_ProfitMargin ~ MA10 + MA20 + RSI_Normalized + Beta + Sharpe_Normalized + Volatility_Normalized,
                    data = pdata, model = "within")
fixed_roe <- plm(log_ROE ~ MA10 + MA20 + RSI_Normalized + Beta + Sharpe_Normalized + Volatility_Normalized,
                 data = pdata, model = "within")

# Fit random effects models using Wallace-Hussain method
random_price <- plm(log_Price ~ MA10 + MA20 + RSI_Normalized + Beta + Sharpe_Normalized + Volatility_Normalized,
                    data = pdata, model = "random", random.method = "walhus")
random_debt <- plm(log_DebtToEquity ~ MA10 + MA20 + RSI_Normalized + Beta + Sharpe_Normalized + Volatility_Normalized,
                   data = pdata, model = "random", random.method = "walhus")
random_profit <- plm(log_ProfitMargin ~ MA10 + MA20 + RSI_Normalized + Beta + Sharpe_Normalized + Volatility_Normalized,
                     data = pdata, model = "random", random.method = "walhus")
random_roe <- plm(log_ROE ~ MA10 + MA20 + RSI_Normalized + Beta + Sharpe_Normalized + Volatility_Normalized,
                  data = pdata, model = "random", random.method = "walhus")

# Perform Hausman test
hausman_price <- phtest(fixed_price, random_price)
hausman_debt <- phtest(fixed_debt, random_debt)
hausman_profit <- phtest(fixed_profit, random_profit)
hausman_roe <- phtest(fixed_roe, random_roe)

# Step 2: Fit Final Models Based on Hausman Test Results
if (hausman_price$p.value < 0.05) {
  cat("Price Model: Fixed Effects Preferred\n")
  final_price <- fixed_price
} else {
  cat("Price Model: Random Effects Preferred\n")
  final_price <- random_price
}

if (hausman_debt$p.value < 0.05) {
  cat("DebtToEquity Model: Fixed Effects Preferred\n")
  final_debt <- fixed_debt
} else {
  cat("DebtToEquity Model: Random Effects Preferred\n")
  final_debt <- random_debt
}

if (hausman_profit$p.value < 0.05) {
  cat("Profit Margin Model: Fixed Effects Preferred\n")
  final_profit <- fixed_profit
} else {
  cat("Profit Margin Model: Random Effects Preferred\n")
  final_profit <- random_profit
}

if (hausman_roe$p.value < 0.05) {
  cat("ROE Model: Fixed Effects Preferred\n")
  final_roe <- fixed_roe
} else {
  cat("ROE Model: Random Effects Preferred\n")
  final_roe <- random_roe
}

# Compute robust standard errors for the final models
robust_price <- coeftest(final_price, vcov = vcovHC(final_price, type = "HC3"))
robust_debt <- coeftest(final_debt, vcov = vcovHC(final_debt, type = "HC3"))
robust_profit <- coeftest(final_profit, vcov = vcovHC(final_profit, type = "HC3"))
robust_roe <- coeftest(final_roe, vcov = vcovHC(final_roe, type = "HC3"))

In [ ]:
# --------------------------
# Phase 5: Diagnostic Testing
# --------------------------
# Lagrange Multiplier tests for panel effects
lm_test_price <- plmtest(final_price)
lm_test_debt <- plmtest(final_debt)
lm_test_profit <- plmtest(final_profit)
lm_test_roe <- plmtest(final_roe)

# Breusch-Pagan tests for heteroskedasticity
bp_test_price <- bptest(final_price)
bp_test_debt <- bptest(final_debt)
bp_test_profit <- bptest(final_profit)
bp_test_roe <- bptest(final_roe)

# --------------------------
# Phase 7: Model Validation and Reporting
# --------------------------
# Extract p-values from robust coefficient tests
extract_p_values <- function(robust_test) {
  p_values <- robust_test[, 4]
  names(p_values) <- rownames(robust_test)
  return(p_values)
}

# Compile model performance metrics
model_performance <- list(
  Price_Model = list(
    R_Squared = summary(final_price)$r.squared,
    Coefficients = coef(robust_price),
    P_Values = extract_p_values(robust_price)
  ),
  Debt_Model = list(
    R_Squared = summary(final_debt)$r.squared,
    Coefficients = coef(robust_debt),
    P_Values = extract_p_values(robust_debt)
  ),
  Profit_Model = list(
    R_Squared = summary(final_profit)$r.squared,
    Coefficients = coef(robust_profit),
    P_Values = extract_p_values(robust_profit)
  ),
  ROE_Model = list(
    R_Squared = summary(final_roe)$r.squared,
    Coefficients = coef(robust_roe),
    P_Values = extract_p_values(robust_roe)
  )
)

In [11]:
# Generate report with clear descriptions
sink("financial_dynamics_report.txt")

# Header
cat("==================================================\n")
cat("Financial Dynamics Analysis Report\n")
cat("==================================================\n\n")

# Hausman Test Results
cat("### Hausman Test Results ###\n")
cat("The Hausman test determines whether fixed effects or random effects models are more appropriate.\n")
cat("A p-value < 0.05 suggests that fixed effects are preferred.\n\n")

cat("1. Price Model:\n")
cat("   - P-Value: ", hausman_price$p.value, "\n")
cat("   - Conclusion: ", ifelse(hausman_price$p.value < 0.05, "Fixed Effects Preferred", "Random Effects Preferred"), "\n\n")

cat("2. DebtToEquity Model:\n")
cat("   - P-Value: ", hausman_debt$p.value, "\n")
cat("   - Conclusion: ", ifelse(hausman_debt$p.value < 0.05, "Fixed Effects Preferred", "Random Effects Preferred"), "\n\n")

cat("3. Profit Margin Model:\n")
cat("   - P-Value: ", hausman_profit$p.value, "\n")
cat("   - Conclusion: ", ifelse(hausman_profit$p.value < 0.05, "Fixed Effects Preferred", "Random Effects Preferred"), "\n\n")

cat("4. ROE Model:\n")
cat("   - P-Value: ", hausman_roe$p.value, "\n")
cat("   - Conclusion: ", ifelse(hausman_roe$p.value < 0.05, "Fixed Effects Preferred", "Random Effects Preferred"), "\n\n")

# Diagnostic Test Results
cat("### Diagnostic Test Results ###\n")
cat("These tests check for panel effects and heteroskedasticity in the models.\n\n")

cat("1. Lagrange Multiplier Test (Price):\n")
cat("   - P-Value: ", lm_test_price$p.value, "\n")
cat("   - Conclusion: ", ifelse(lm_test_price$p.value < 0.05, "Panel Effects Detected", "Pooled Model Suggested"), "\n\n")

cat("2. Lagrange Multiplier Test (DebtToEquity):\n")
cat("   - P-Value: ", lm_test_debt$p.value, "\n")
cat("   - Conclusion: ", ifelse(lm_test_debt$p.value < 0.05, "Panel Effects Detected", "Pooled Model Suggested"), "\n\n")

cat("3. Lagrange Multiplier Test (Profit Margin):\n")
cat("   - P-Value: ", lm_test_profit$p.value, "\n")
cat("   - Conclusion: ", ifelse(lm_test_profit$p.value < 0.05, "Panel Effects Detected", "Pooled Model Suggested"), "\n\n")

cat("4. Lagrange Multiplier Test (ROE):\n")
cat("   - P-Value: ", lm_test_roe$p.value, "\n")
cat("   - Conclusion: ", ifelse(lm_test_roe$p.value < 0.05, "Panel Effects Detected", "Pooled Model Suggested"), "\n\n")

cat("5. Breusch-Pagan Test (Price):\n")
cat("   - P-Value: ", bp_test_price$p.value, "\n")
cat("   - Conclusion: ", ifelse(bp_test_price$p.value < 0.05, "Heteroskedasticity Detected", "Homoskedasticity Assumed"), "\n\n")

cat("6. Breusch-Pagan Test (DebtToEquity):\n")
cat("   - P-Value: ", bp_test_debt$p.value, "\n")
cat("   - Conclusion: ", ifelse(bp_test_debt$p.value < 0.05, "Heteroskedasticity Detected", "Homoskedasticity Assumed"), "\n\n")

cat("7. Breusch-Pagan Test (Profit Margin):\n")
cat("   - P-Value: ", bp_test_profit$p.value, "\n")
cat("   - Conclusion: ", ifelse(bp_test_profit$p.value < 0.05, "Heteroskedasticity Detected", "Homoskedasticity Assumed"), "\n\n")

cat("8. Breusch-Pagan Test (ROE):\n")
cat("   - P-Value: ", bp_test_roe$p.value, "\n")
cat("   - Conclusion: ", ifelse(bp_test_roe$p.value < 0.05, "Heteroskedasticity Detected", "Homoskedasticity Assumed"), "\n\n")

# Model Performance
cat("### Model Performance ###\n")
cat("This section provides the R-squared values, coefficients, and p-values for the final models.\n\n")

cat("1. Price Model:\n")
cat("   - R-Squared: ", model_performance$Price_Model$R_Squared, "\n")
cat("   - Coefficients:\n")
print(model_performance$Price_Model$Coefficients)
cat("\n   - P-Values:\n")
print(model_performance$Price_Model$P_Values)
cat("\n")

cat("2. DebtToEquity Model:\n")
cat("   - R-Squared: ", model_performance$Debt_Model$R_Squared, "\n")
cat("   - Coefficients:\n")
print(model_performance$Debt_Model$Coefficients)
cat("\n   - P-Values:\n")
print(model_performance$Debt_Model$P_Values)
cat("\n")

cat("3. Profit Margin Model:\n")
cat("   - R-Squared: ", model_performance$Profit_Model$R_Squared, "\n")
cat("   - Coefficients:\n")
print(model_performance$Profit_Model$Coefficients)
cat("\n   - P-Values:\n")
print(model_performance$Profit_Model$P_Values)
cat("\n")

cat("4. ROE Model:\n")
cat("   - R-Squared: ", model_performance$ROE_Model$R_Squared, "\n")
cat("   - Coefficients:\n")
print(model_performance$ROE_Model$Coefficients)
cat("\n   - P-Values:\n")
print(model_performance$ROE_Model$P_Values)
cat("\n")

# Footer
cat("==================================================\n")
cat("End of Report\n")
cat("==================================================\n")

sink()





Price Model: Random Effects Preferred
DebtToEquity Model: Random Effects Preferred
Profit Margin Model: Random Effects Preferred
ROE Model: Random Effects Preferred


**Results:**

Hausman Test Results: Random effects models are preferred for all dependent variables.

Diagnostic Test Results: Panel effects are detected for Debt-to-Equity, Profit Margin, and ROE, but not for Price. Heteroskedasticity is detected for Price, but not for the other models.

Model Performance:

Price Model: R-squared = 0.972, all predictors are highly significant.

Debt-to-Equity Model: R-squared = 0.041, only Beta and Sharpe are significant.

Profit Margin Model: R-squared = 0.088, none of the predictors are significant.

ROE Model: R-squared = 0.118, only Beta is significant.



---

Model Performance:
Price Model:

R-Squared: 0.972155 (Adjusted: 0.9714703)

The model explains 97.2% of the variance in Price, which is exceptionally high. This suggests that the predictors (MA10, MA20, RSI, Beta, Sharpe, Volatility) are highly effective in explaining Price movements.

Significant Predictors:

MA10 (p = 1.861384e-04), MA20 (p = 2.681183e-29), RSI_Normalized (p = 4.888511e-15), Beta (p = 7.981305e-80), Sharpe_Normalized (p = 5.903647e-90), Volatility_Normalized (p = 6.936173e-170)

All predictors except the intercept are highly significant, indicating that technical and risk metrics strongly influence Price.

Interpretation:

Higher Beta and Volatility are associated with higher Prices, suggesting that riskier stocks tend to have higher prices.

Sharpe_Normalized (risk-adjusted returns) also positively impacts Price, indicating that better risk-adjusted performance leads to higher prices.

MA10 and MA20 have mixed effects, with MA10 negatively impacting Price and MA20 positively impacting it.

DebtToEquity Model:

R-Squared: 0.04113767 (Adjusted: 0.01755909)

The model explains only 4.1% of the variance in DebtToEquity, which is very low. This suggests that the predictors are not effective in explaining DebtToEquity.

Significant Predictors:

Beta (p = 0.001433645), Sharpe_Normalized (p = 0.003898491)

Only Beta and Sharpe_Normalized are significant, indicating that higher risk (Beta) and better risk-adjusted returns (Sharpe) are associated with higher DebtToEquity ratios.

Interpretation:

Companies with higher Beta (market risk) tend to have higher DebtToEquity ratios, possibly because they rely more on debt financing.

Sharpe_Normalized also positively impacts DebtToEquity, suggesting that companies with better risk-adjusted returns may take on more debt.

Profit Margin Model:

R-Squared: 0.0877602 (Adjusted: 0.06532808)

The model explains only 8.8% of the variance in Profit Margin, which is low. This suggests that the predictors are not effective in explaining Profit Margin.

Significant Predictors:

None of the predictors are significant at the 5% level.

Interpretation:

The lack of significant predictors suggests that technical and risk metrics do not strongly influence Profit Margin. Other factors, such as operational efficiency or cost structures, may be more important.

ROE Model:

R-Squared: 0.1183236 (Adjusted: 0.09664305)

The model explains 11.8% of the variance in ROE, which is relatively low. This suggests that the predictors are not highly effective in explaining ROE.

Significant Predictors:

Beta (p = 0.003664649)

Only Beta is significant, indicating that higher market risk is associated with higher ROE.

Interpretation:

Companies with higher Beta (market risk) tend to have higher ROE, possibly because they take on more risk to achieve higher returns.